# Импорт модулей и библиотек

Установим библиотеку polara

In [1]:
# pip install --no-cache-dir --upgrade git+https://github.com/evfro/polara.git#egg=polara

In [1]:
import pickle
import pandas as pd
import scipy
import numpy as np

# Scipy
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

import implicit
from lightfm import LightFM
from lightfm.data import Dataset

# Implicit
from implicit.nearest_neighbours import CosineRecommender
from implicit.als import AlternatingLeastSquares

C:\Users\isaev\miniconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


# Preprocessing

In [67]:
all_years_df = pd.DataFrame([])
for year in range(2015, 2022):
    with open(f'Data\df_{year}_cleaned.pickle', "rb") as file:
        df_years = pickle.load(file)
    all_years_df = all_years_df.append(df_years, ignore_index=True)
        
print(all_years_df.shape)
all_years_df.head()

(6156534, 7)


,datetime,ticker,deals,price,user,cf,date
0,2015-11-17 18:36:00,NLMK,1090,71.37,1_62009,-77793.30,2015-11-17
1,2015-11-17 18:36:00,NLMK,270,71.39,1_62009,-19275.30,2015-11-17
2,2015-11-17 18:37:00,NLMK,60,71.37,1_62009,-4282.20,2015-11-17
3,2015-12-09 12:55:00,MTLR,1,62.65,1_62009,-62.65,2015-12-09
4,2015-12-09 12:56:00,MTLR,1,62.65,1_62009,-62.65,2015-12-09


Проверим на наличие пропущенных значений

In [3]:
all_years_df.isnull().sum()

datetime    0
ticker      0
deals       0
price       0
user        0
cf          0
date        0
dtype: int64

*Проверим на наличие дубликотов*

In [4]:
all_years_df = all_years_df.drop_duplicates().reset_index(drop=True)

*Нулей нет, дубликтов нет -  можно работать дальше*

In [92]:
df = all_years_df.groupby(['ticker', 'user'], as_index=False)['deals', 'price', 'cf'].sum()
df.head()

C:\Users\isaev\AppData\Local\Temp/ipykernel_9412/2509338410.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df = all_years_df.groupby(['ticker', 'user'], as_index=False)['deals', 'price', 'cf'].sum()


,ticker,user,deals,price,cf
0,AFKS,1_100013,-100,19.500,1950.0
1,AFKS,1_100018,1000,18.980,-18980.0
2,AFKS,1_100033,0,501.830,156.0
3,AFKS,1_100049,0,221.835,-875.0
4,AFKS,1_100050,0,156.830,674.0


In [93]:
#Создание новых колонок с ID тикеров и юзеров

tick2ind = {ticker:i for i, ticker in enumerate(df.ticker.unique())}
ind2tick = {val:key for key, val in tick2ind.items()}

df['new_ticker'] = df['ticker'].apply(lambda x: tick2ind[x])

user2ind = {user:i for i, user in enumerate(df.user.unique())}
ind2user = {val:key for key, val in user2ind.items()}

df['new_user'] = df['user'].apply(lambda x: user2ind[x])


print(df.shape)
df.sample(10)

(246857, 7)


,ticker,user,deals,price,cf,new_ticker,new_user
222783,TATNP,1_247232,-250,3675.800,134765.0,61,2948
31139,CHMF,1_63148,-130,2178.500,88312.0,7,15435
110215,MOEX,1_286349,-3000,2209.890,510004.6,29,33247
201469,SBERP,1_99772,-100,336.510,9405.0,54,29630
246695,YNDX,1_91175,0,12384.000,1070.0,69,15942
42298,FXCN,1_245437,1,4081.000,-4081.0,14,2711
48720,GAZP,1_240603,-990,11720.140,261576.7,18,7167
208273,SNGS,1_62121,-300,35.795,10738.5,58,4809
186166,SBER,1_253540,-900,486.870,219087.0,53,30408
167850,ROSN,1_87760,0,48702.750,-181665.5,48,10885


### Создаем словарь: ID юзера и его сделки

In [94]:
def user_deals(df, data):    
    
    """
    Функция получает на вход столбец датафрейма с айдишниками юзеров
    
    На выходе: словарь с ключами - ID юзера, а значения - все его сделки
    """
    
    #df = pd.DataFrame()
    new_user = {}
    users=set(data.values)
    
    for user in users:
        deals = df[(data==user)]
        if len(deals)==1:
            continue
        else:
            new_user[user] = deals    
            
    return new_user

# Train_test_split

In [95]:
def train_test_split(train_size=0.7, test_size=0.3):
    
    train = pd.DataFrame()
    test = pd.DataFrame()
    dict_user = user_deals(df, df["new_user"])
    
    for key in dict_user.keys():
        deals_count = len(dict_user[key])
        to_train = round(deals_count*train_size)
        to_test = round(deals_count*test_size)
        
        # Если деление произойдет не по тем пропорциям
        # while (to_train+to_test)>deals_count:
        #    to_train-=1
            
        train = pd.concat([train, dict_user[key][:to_train]])
        test = pd.concat([test, dict_user[key][-to_test:]])
    return (train, test)

In [96]:
%%time
train, test = train_test_split(train_size=0.7, test_size=0.3)

Wall time: 7min 19s


In [97]:
print("Train size", train.shape)
print("Test size", test.shape)

Train size (165960, 7)
Test size (74692, 7)


In [98]:
train.head(5)

,ticker,user,deals,price,cf,new_ticker,new_user
0,AFKS,1_100013,-100,19.50,1950.0,0,0
136478,NVTK,1_100013,0,1316.80,122.0,38,0
178502,SBER,1_100013,0,287.31,27.8,53,0
2,AFKS,1_100033,0,501.83,156.0,0,2
44199,GAZP,1_100033,-70,9702.56,11312.1,18,2


In [12]:
test.head()

,ticker,user,deals,new_ticker,new_user
205306,SNGS,1_100013,1,58,0
172011,RTKM,1_100033,2,50,2
205307,SNGS,1_100033,43,58,2
175690,RUAL,1_100049,2,52,3
225934,TGKA,1_100049,20,63,3


# Создание sparse матрицы с помощью метода CSR

In [13]:
# Train dataset
n_rows_train = train["new_user"].nunique()
n_cols_train = train["new_ticker"].nunique()

row_train = train["new_user"] #user
col_train = train["new_ticker"] #ticker
data_train = train["deals"].astype(float) #deals

# Test dataset
n_rows_test = test["new_user"].nunique()
n_cols_test = test["new_ticker"].nunique()

row_test = test["new_user"] #user
col_test = test["new_ticker"] #ticker
data_test = test["deals"].astype(float) #deals

sample_matr_train = csr_matrix((data_train, (row_train, col_train)))

sample_matr_test = csr_matrix((data_test, (row_test, col_test)))

In [14]:
sample_matr_train.shape

(40601, 69)

In [15]:
sample_matr_test.shape

(40601, 70)

# Метрики

Precision можно интепретировать как долю релевантных рекомендаций, мы дополнительно считаем среднее по всем пользователям

Precision = TP / (TP + FP), где TP - фильмы из тестовой выборки, совпавшие с рекомендациями, а TP + FP - это все рекомендованные объекты, то есть K - количество предсказанных объектов

In [16]:
def precision(test_data, predict_data):

    '''
     На вход:
     - test_data - тестовый набор данных
     - predict_data - предсказания

     k берется таким, для которого мы рассчитывали рекомендации (длина рекомендаций)

     На выходе:
         Средняя метрика precision@k по всем пользователям
    '''
    
    sum_precision = 0
    
    # вычисляем метрику по каждому пользователю
    for user in test_data.index:
        try:
            TP = len(set(predict_data['ticker'][user]).intersection(set(test_data['ticker'][user])))   
            k = len(predict_data['ticker'][user])
            
        except TypeError:
            TP = 0
            K = 0
    
        sum_precision += TP / k

    # находим среднее по пользователям
    return sum_precision / len(test_data.index.unique())

Recall интерпретируется как доля релевантных объектов, попавших в рекомендации, мы дополнительно считаем среднее по всем пользователям

Recall = TP / (TP + FN), где TP - фильмы из тестовой выборки, совпавшие с рекомендациями, а TP + FN - это все объекты из тестовой выборки

In [17]:
def mean_recall(test_data, predict_data):

    '''
     На вход подается:
         - test_data - тестовый набор данных
         - predict_data - предсказания

     k берется таким, для которого мы рассчитывали рекомендации (длина рекомендаций)

     На выходе:
         Средняя метрика recall@k по всем пользователям
    '''

    sum_recall = 0

    # вычисляем метрику по каждому пользователю
    for user in test_data.index.unique():
        
        TP = len(set(predict_data['ticker'][user]).intersection(set(test_data['ticker'][user])))
        TP_FN = len(test_data['ticker'][user])

        sum_recall += TP / TP_FN

    # находим среднее по пользователям
    return sum_recall / len(test_data.index.unique())

Mean Reciprocal Rank - средний обратный ранк первого правильного вхождения рекомендации в тестовые данные

In [18]:
def MRR(test_data, predict_data):

    '''
    RR - обратный ранг: 1 / индеск первого вхождения рекомендации в тест

    На вход подается:
        - test_data - тестовый набор данных
        - predict_data - предсказания

    k берется таким, для которого мы рассчитывали рекомендации (длина рекомендаций)

    На выходе:
        Метрика MRR@k по всем пользователям
    '''

    sum_rr = 0

    # находим порядковый номер первого вхождения правильной рекомендации в тест
    for user in test_data.index.unique():
        
        rank = [1 + list(predict_data['ticker'][user]).index(x) for x in \
                set(predict_data['ticker'][user]).intersection(set(test_data['ticker'][user]))]

    # находим обратный ранк, если попадания нет, то зануляем
    if len(rank) == 0:
        rr = 0
    else:
        rr = 1 / min(rank)

    sum_rr += rr

    # находим среднее по пользователям
    return sum_rr / len(test_data.index.unique())

Average Precision дополнительно вознаграждает нас за предварительну загрузку рекомендаций, которые будут правильными, то есть здесь, в отличие от Precision, учитывается порядок выдаваемых рекомендаций и их точность отдельно. MAP - это среднее AP по всем пользователям

In [19]:
def mean_average_precision_at_k(test_data, predict_data):

    '''
    Mean Average Precision - средняя точность по пользователям

    На вход:
        - test_data - тестовый набор данных
        - predict_data - предсказания

    k берется таким, для которого мы рассчитывали рекомендации (длина рекомендаций)

    На выходе:
        Метрика MAP@k по всем пользователям
    '''

    sum_ap = 0

    # итерируемся по пользователям
    for user in test_data.index.unique():

        num_hits = 0
        score = 0

        for i, p in enumerate(predict_data['ticker'][user]):

            if p in test_data['ticker'][user] and p not in predict_data['ticker'][user][:i]:
                num_hits += 1
                score += num_hits / (i + 1)

        sum_ap += score / min(len(test_data['ticker'][user]), len(predict_data['ticker'][user]))

    # находим среднее по пользователям
    return sum_ap / len(test_data.index.unique())

### Сингулярное разложение матрицы

In [20]:
# Train
U_train, S_train, VT_train = svds(sample_matr_train, k=66)

# Test
U_test, S_test, VT_test = svds(sample_matr_test, k=60)

In [21]:
print('Train size: ', U_train.shape, '|', S_train.shape, '|', VT_train.shape)
print('Test size: ', U_test.shape, '|', S_test.shape, '|', VT_test.shape)

Train size:  (40601, 66) | (66,) | (66, 69)
Test size:  (40601, 60) | (60,) | (60, 70)


In [22]:
S_train_diag = np.diag(S_train)
print(S_train_diag.shape)

S_test_diag = np.diag(S_test)
print(S_test_diag.shape)

(66, 66)
(60, 60)


### Dense матрица

In [23]:
matrics_singular_train = U_train @ S_train_diag @ VT_train

matrics_singular_test = U_test @ S_test_diag @ VT_test

print(matrics_singular_train.shape)
print(matrics_singular_test.shape)

(40601, 69)
(40601, 70)


In [24]:
sp_matr_singular_train = csr_matrix(matrics_singular_train)

sp_matr_singular_test = csr_matrix(matrics_singular_test) 

print(sp_matr_singular_train.shape)
print(sp_matr_singular_test.shape)

(40601, 69)
(40601, 70)


In [25]:
error_train = scipy.sparse.linalg.norm(sample_matr_train - sp_matr_singular_train) / scipy.sparse.linalg.norm(sample_matr_train)

error_test = scipy.sparse.linalg.norm(sample_matr_test - sp_matr_singular_test) / scipy.sparse.linalg.norm(sample_matr_test)

print('Train error = ',error_train)
print('Test error = ',error_test)

Train error =  0.0008076185877393496
Test error =  0.00040341602181824575


In [26]:
df_VT_train = pd.DataFrame(VT_train)

df_VT_test = pd.DataFrame(VT_test)

print(df_VT_train.shape)
print(df_VT_test.shape)

(66, 69)
(60, 70)


# Косинусное расстояние

In [27]:
def my_cos_dist(x, y):
    
    '''
    На вход:
        x - матрица 1
        y - матрица 2
    
    На выходе:
        Вектор с вычисленным косинусным расстоянием между обеими матрицы
    '''
    
    num = x.T @ y
    norm_x = np.linalg.norm(x)
    norm_y = np.linalg.norm(y)
    result = num / norm_x*norm_y
    
    return result

In [28]:
def correcaltion_func(x, y):
    
    '''
    На вход:
        x - матрица 1
        y - матрица 2
    
    На выходе:
        Вектор с вычисленной корреляцией между обеими матрицы
    '''
    
    result = np.corrcoef(x, y)[1, 0]
    
    return result

In [29]:
def compare_stocks(ticker_to_compare, emb_matrix_df, ticker2id_dict, id2ticker_dict=dict(), 
                   dist_func = {'my_cos_dist': my_cos_dist}, col_to_sort = ''):
    
    '''
    На вход:
        emb_matrix_df - матрица эмбедингов акций
        ticker2id_dict - матрица 
    
    На выходе:
        Вектор с вычисленным косинусным расстоянием между обеими матрицы
    '''
    
    if len(id2ticker_dict) == 0:
        id2ticker_dict = {val:key for key, val in ticker2id_dict.items()}
        
    if col_to_sort == '':
        col_to_sort = list(dist_func.keys())[0]
    
    id_to_compare = ticker2id_dict[ticker_to_compare]
    vec_to_compare_0 = emb_matrix_df[id_to_compare].values
    stock_to_compare = emb_matrix_df[id_to_compare]
    
    other_indexes = emb_matrix_df.columns.tolist()
    other_indexes.remove(id_to_compare)
    
    result_array = np.zeros((len(other_indexes), len(dist_func)))
    for i, stock_id in enumerate(other_indexes):
        vec_to_compare_1 = emb_matrix_df[stock_id].values
        
        res_line = []
        for key_func in dist_func:
            res_el = dist_func[key_func](vec_to_compare_0, vec_to_compare_1)
            res_line.append(res_el)
            
        result_array[i, :] = res_line
    
    true_name_stocks = [id2ticker_dict[id_] for id_ in other_indexes]
    result_df = pd.DataFrame(result_array, columns = dist_func.keys(), index = true_name_stocks)
    result_df = result_df.sort_values(by=col_to_sort, ascending=False)
    return result_df

In [30]:
my_dist_func = {'my_cos_dist': my_cos_dist,
               'correlation': correcaltion_func}

ratio_train = compare_stocks('GAZP', df_VT_train, tick2ind, ind2tick, my_dist_func)
ratio_train = ratio_train.sort_values('correlation', ascending=False)

ratio_test = compare_stocks('YNDX', df_VT_test, tick2ind, ind2tick, my_dist_func)
ratio_test = ratio_test.sort_values('correlation', ascending=False)

# Модели

In [31]:
model_names = ['CosineRecommender', 'AlternatingLeastSquares']

metrics = {'Precision@k': precision, 'Recall@k': mean_recall, 'Mean Reciprocal Rank': MRR, 
           'Mean Average Precision': mean_average_precision_at_k}

results = pd.DataFrame([])

In [32]:
def calculate_metrics(test_data, predict_data, metrics={}):
    
    '''
    На вход:
        test_data - тестовый набор данных
        predict_data - рекомендации для пользователя
    
    На выходе:
        Вычисленная таблица с результатами 
    '''
    
    if len(metrics) == 0:
        raise ValueError('metrics are empty')
        
    results = dict()
    for key in metrics:
        results[key] = metrics[key](test_data, predict_data)
    
    return results

# Implicit

### Item-to-item модели
Векорное представление для объекта - весь столбец из матрицы взаимодействий (user_item) матрица

Процесс построения рекомендаций - поиск похожих объектов (по косинусной близости) для всех объектов, с которыми пользователь уже взаимодействовал и выдача топа из этого списка

Пространство для тюнинга - взвешние на уровне таблицы, взвешивание элементов матрицы (на уровне моделей), кол-во соседей K

Этим модели для Item-to-item подхода ожидают на вход матрицу в ориентации item_user, поэтому в fit передаем train_mat.T

In [33]:
cosine_model = CosineRecommender(K=20)
cosine_model.fit(sample_matr_train)

  0%|          | 0/69 [00:00<?, ?it/s]

Параметр K влияет на максимальную выдачу топа, поэтому N для таких моделей желательно указывать меньше K

In [34]:
def recommend_tick(user, model, train_matrix, N):
    user_id = user2ind[user]

    recs = model.recommend(user_id, train_matrix[user_id], N=N, filter_already_liked_items=True)
    
    res = [ind2tick[ticker_ids] for ticker_ids in recs[0]]
    return res

In [35]:
def find_similar(user, model, train_matrix, N):
    user_id = user2ind[user]

    recs = model.recommend(user_id, train_matrix[user_id], N=N, filter_already_liked_items=True)
    
    res = recs[1]
    return res

In [36]:
top_N = 10
df_recs_cossim = pd.DataFrame({'user': test['user'].unique()})

df_recs_cossim['ticker'] = df_recs_cossim['user'].apply(lambda x: 
                                                        recommend_tick(x, cosine_model, sample_matr_train, top_N))

df_recs_cossim['similar'] = df_recs_cossim['user'].apply(lambda x: find_similar(x, cosine_model, sample_matr_train, top_N))
df_recs_cossim.head()

,user,ticker,similar
0,1_100013,"[NLMK, IRAO, MGNT, GMKN, AFLT, GAZP, MAGN, HYD...","[0.5349589623089654, 0.5436748159591326, 0.614..."
1,1_100033,"[RSTI, NLMK, GMKN, MOEX, MGNT, NVTK, ROSN, ALR...","[11.335317989439316, 11.562178263146457, 13.11..."
2,1_100049,"[ALRS, CHMF, NMTP, OGKB, AFLT, MSNG, PHOR, PIK...","[14.448589958341046, 15.343560451662828, 21.08..."
3,1_100050,"[RASP, POLY, LSRG, QIWI, PLZL, NLMK, NVTK, RST...","[6.944676828099718, 7.280567977000016, 12.9769..."
4,1_100093,"[MAIL, ROSN, FEES, HYDR, GAZP, OGKB, RSTI, GMK...","[14.779178451092061, 20.186533130597443, 18.45..."


In [37]:
df_recs_cossim = df_recs_cossim.explode(['similar', 'ticker'])
df_recs_cossim['rank'] = df_recs_cossim.groupby('user').cumcount() + 1
df_recs_cossim.head(top_N + 2)

,user,ticker,similar,rank
0,1_100013,NLMK,0.534959,1
0,1_100013,IRAO,0.543675,2
0,1_100013,MGNT,0.614498,3
0,1_100013,GMKN,0.595059,4
0,1_100013,AFLT,0.79658,5
0,1_100013,GAZP,0.616513,6
0,1_100013,MAGN,0.880075,7
0,1_100013,HYDR,0.882497,8
0,1_100013,ROSN,0.70815,9
0,1_100013,CHMF,0.89358,10


In [38]:
user2ind = {user:i for i, user in enumerate(test['user'].unique())}

test.index = test['user'].apply(lambda x: user2ind[x])

Вычисляем результаты

In [39]:
result_cossim = calculate_metrics(test, df_recs_cossim, metrics=metrics)  
results = results.append(result_cossim, ignore_index=True)

## ALS

Векорное представление - на основе разложения матрицы взамиодействий (с весами) в произведение двух матриц. Одна матрица размерности [users x factors] содержит векторные представления всех пользователей, другая матрица [items x factors] для объектов

Процесс построения рекомендаций - для конкретного пользователя ищутся такие объекты, что скалярное произведение их векторов максимально.

Пространство для тюнинга

* взвешивание на уровне таблицы
* factors - размерность итоговых векторов (обычно степени 2-ки, от 16 до 256)
* iterations - кол-во итераций (от 10 до 100)
* regularization - регуляризация векторов (степени 10-ки, от 0.0001 до 1)
* Есть возможность использовать GPU через use_gpu=True и ApproximateNearestNeighbors (на уровне модели)

In [40]:
als_model = AlternatingLeastSquares(factors=32, iterations=40)
als_model.fit(sample_matr_train) 

C:\Users\isaev\miniconda3\lib\site-packages\implicit\utils.py:31: UserWarning: Intel MKL BLAS detected. Its highly recommend to set the environment variable 'export MKL_NUM_THREADS=1' to disable its internal multithreading
  warnings.warn(


  0%|          | 0/40 [00:00<?, ?it/s]

In [41]:
top_N = 10
df_recs_als = pd.DataFrame({'user': test['user'].unique()})

df_recs_als['ticker'] = df_recs_als['user'].apply(lambda x: recommend_tick(x, als_model, sample_matr_train, top_N))
df_recs_als['similar'] = df_recs_als['user'].apply(lambda x: find_similar(x, als_model, sample_matr_train, top_N))

df_recs_als.head()

,user,ticker,similar
0,1_100013,"[AGRO, RUAL, SBERP, FIVE, SMLT, IRAO, PIKK, TR...","[0.13501224, 0.11469802, 0.092735246, 0.080921..."
1,1_100033,"[ENPG, DSKY, CHMF, CBOM, BSPB, BELU, ALRS, AGR...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1_100049,"[RSTI, RASP, RUAL, RTKMP, DSKY, POLY, MAGN, LS...","[0.30811054, 0.24100569, 0.2000211, 0.19397289..."
3,1_100050,"[NMTP, LNTA, MSNG, DSKY, RASP, TATN, SNGSP, LS...","[1.0029626, 0.6611083, 0.48966327, 0.436554, 0..."
4,1_100093,"[DSKY, RUAL, AGRO, FIVE, RASP, MRKC, RSTI, MSN...","[0.31526366, 0.20684677, 0.18116432, 0.1730029..."


In [42]:
df_recs_als = df_recs_als.explode(['ticker', 'similar'])
df_recs_als['rank'] = df_recs_als.groupby('user').cumcount() + 1
df_recs_als.head(top_N + 2)

,user,ticker,similar,rank
0,1_100013,AGRO,0.135012,1
0,1_100013,RUAL,0.114698,2
0,1_100013,SBERP,0.092735,3
0,1_100013,FIVE,0.080921,4
0,1_100013,SMLT,0.072831,5
0,1_100013,IRAO,0.070438,6
0,1_100013,PIKK,0.055413,7
0,1_100013,TRNFP,0.055395,8
0,1_100013,PHOR,0.053897,9
0,1_100013,VTBR,0.050878,10


Вычисляем результаты

In [43]:
result_als = calculate_metrics(test, df_recs_als, metrics=metrics)  
results = results.append(result_als, ignore_index=True)

In [44]:
results.index = model_names
results

,Precision@k,Recall@k,Mean Reciprocal Rank,Mean Average Precision
CosineRecommender,0.070347,0.043645,0.0,0.004558
AlternatingLeastSquares,0.109128,0.079795,0.0,0.004080


Выгружаем данные

In [99]:
user2ind = {user:i for i, user in enumerate(train['user'].unique())}

train['new'] = train['user'].apply(lambda x: user2ind[x])
train.index = train['new']

train.to_excel('Train_dataset.xlsx')
results.to_excel('Metrics.xlsx')

In [52]:
df_to_csv = pd.DataFrame()
df_to_csv[['user']] = df_recs_als[['user']]
df_to_csv[['Model_1', 'Similar_1']] = df_recs_cossim[['ticker', 'similar']]
df_to_csv[['Model_2', 'Similar_2']] = df_recs_als[['ticker', 'similar']]

In [64]:
df_to_csv.to_excel('Recommendation.xlsx')